<a href="https://colab.research.google.com/github/Maryam-youssef/Fraud-Detection/blob/main/Preprocessing_FeatureEngineering_GANs_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

In [ ]:
df = pd.read_csv('archive\Synthetic_Financial_datasets_log.csv')

<>:1: SyntaxWarning: invalid escape sequence '\S'
<>:1: SyntaxWarning: invalid escape sequence '\S'
C:\Users\mario\AppData\Local\Temp\ipykernel_16128\3510858160.py:1: SyntaxWarning: invalid escape sequence '\S'
  df = pd.read_csv('archive\Synthetic_Financial_datasets_log.csv')


In [ ]:
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Label Encoding for the 'type' column
label_encoder = LabelEncoder()
df['type'] = label_encoder.fit_transform(df['type'])

# Feature Engineering

# Drastic change in balance after the transaction
df['balance_change_orig'] = df['oldbalanceOrg'] - df['newbalanceOrig']

# Flagging transactions where the sender's balance is drained
df['drained_orig_balance'] = np.where((df['newbalanceOrig'] == 0) & (df['amount'] > 0), 1, 0)

# Flagging transactions where the destination had zero balance before the transaction
df['zero_balance_dest'] = np.where(df['oldbalanceDest'] == 0, 1, 0)
df['zero_balance_orig'] = np.where(df['oldbalanceOrg'] == 0, 1, 0)

# Dropping low-correlation columns
df.drop(columns=['step'], inplace=True)
df.drop(columns=['nameOrig'], inplace=True)
df.drop(columns=['nameDest'], inplace=True)
df.drop(columns=['newbalanceDest'], inplace=True)

In [ ]:
df.head()

,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,isFraud,isFlaggedFraud,balance_change_orig,drained_orig_balance,zero_balance_dest,zero_balance_orig
0,3,9839.64,170136.0,160296.36,0.0,0,0,9839.64,0,1,0
1,3,1864.28,21249.0,19384.72,0.0,0,0,1864.28,0,1,0
2,4,181.00,181.0,0.00,0.0,1,0,181.00,1,1,0
3,1,181.00,181.0,0.00,21182.0,1,0,181.00,1,0,0
4,3,11668.14,41554.0,29885.86,0.0,0,0,11668.14,0,1,0


In [ ]:
df.corrwith(df["isFraud"])


type                    0.020833
amount                  0.076688
oldbalanceOrg           0.010154
newbalanceOrig         -0.008148
oldbalanceDest         -0.005885
isFraud                 1.000000
isFlaggedFraud          0.044109
balance_change_orig     0.362472
drained_orig_balance    0.029843
zero_balance_dest       0.016471
zero_balance_orig      -0.024874
dtype: float64

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Input
from keras.optimizers import Adam
from keras.metrics import Precision, Recall
from tensorflow.keras import backend as K

# Prepare dataset
X = df.drop('isFraud', axis=1)
y = df['isFraud']

# Apply Standard Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Splitting the data using stratified train/test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Combining the features and labels for the training set
train_df = pd.DataFrame(X_train, columns=X.columns)
train_df['isFraud'] = y_train.values

# Separate isFraud and non-isFraud data
isFraud_data = train_df[train_df['isFraud'] == 1].drop('isFraud', axis=1).values
non_isFraud_data = train_df[train_df['isFraud'] == 0].drop('isFraud', axis=1).values

# Calculate the number of synthetic isFraud samples needed
num_real_isFraud = len(isFraud_data)
#num_synthetic_samples = len(non_isFraud_data) - num_real_isFraud
num_synthetic_samples = 1000000
print("# of non-isFraud: ", len(non_isFraud_data))
print("# of Real isFraud: ", num_real_isFraud)
print("# of Synthetic isFraud required: ", num_synthetic_samples)

# Define the generator network
def build_generator(latent_dim, output_dim):
    model = Sequential()
    model.add(Dense(64, input_shape=(latent_dim,)))
    model.add(Dense(128, activation='sigmoid'))
    model.add(Dense(output_dim, activation='sigmoid'))
    return model

# Define the discriminator network
def build_discriminator(input_dim):
    model = Sequential()
    model.add(Dense(128, activation='sigmoid', input_dim=input_dim))
    model.add(Dense(128, activation='sigmoid'))
    model.add(Dense(1, activation='sigmoid'))
    return model

# Custom loss function for the generator
def generator_loss_log_d(y_true, y_pred):
    return -K.mean(K.log(y_pred + K.epsilon()))


# Dimensionality of the input noise for the generator
latent_dim = 32

# Build the generator and discriminator
generator = build_generator(latent_dim, isFraud_data.shape[1])
discriminator = build_discriminator(isFraud_data.shape[1])

# Compile the discriminator with cross-entropy loss and precision/recall metrics
discriminator.compile(optimizer=Adam(learning_rate=0.0002, beta_1=0.5), loss='binary_crossentropy', metrics=[Precision(), Recall()])

# Build and compile the GANs main optimization loop
def build_gan(generator, discriminator):
    discriminator.trainable = False  # Freeze discriminator when training the generator
    model = Sequential()
    model.add(generator)
    model.add(discriminator)
    model.compile(optimizer=Adam(learning_rate=0.0002, beta_1=0.5), loss=generator_loss_log_d)
    return model

# Compile the GAN model
gan = build_gan(generator, discriminator)

# Set training parameters
epochs = 1000
batch_size = 32

# Training loop for GANs
for epoch in range(epochs):
    # Train the discriminator
    discriminator.trainable = True
    generator.trainable = False

    # Random sampling from real isFraud data
    real_isFraud_samples = isFraud_data[np.random.randint(0, num_real_isFraud, batch_size)]

    # Generate fake isFraud samples using the generator
    noise = np.random.normal(0, 1, size=(batch_size, latent_dim))
    fake_isFraud_samples = generator.predict(noise)

    # Create labels for real and fake isFraud samples
    real_labels = np.ones((batch_size, 1))
    fake_labels = np.zeros((batch_size, 1))

    # Train the discriminator on real and fake isFraud samples
    d_loss_real = discriminator.train_on_batch(real_isFraud_samples, real_labels)
    d_loss_fake = discriminator.train_on_batch(fake_isFraud_samples, fake_labels)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # Train the generator
    discriminator.trainable = False
    generator.trainable = True

    # Generate noise and valid labels for generator training
    noise = np.random.normal(0, 1, size=(batch_size, latent_dim))
    valid_labels = np.ones((batch_size, 1))

    # Train the generator to fool the discriminator
    g_loss = gan.train_on_batch(noise, valid_labels)

    # Print progress every 100 epochs
    if epoch % 100 == 0:
        print(f"Epoch: {epoch} - D Loss: {d_loss} - G Loss: {g_loss}")

# Generate synthetic isFraud data




NameError: name 'df' is not defined

In [ ]:
import pandas as pd
import numpy as np

noise = np.random.normal(0, 1, size=(num_synthetic_samples, latent_dim))
synthetic_isFraud_data = generator.predict(noise)

# Convert synthetic isFraud data into a DataFrame
import pandas as pd
import numpy as np

# Convert NumPy array
column_names = ['type','amount','oldbalanceOrg','newbalanceOrig','oldbalanceDest','isFlaggedFraud','balance_change_orig','drained_orig_balance','zero_balance_dest','zero_balance_orig']

synthetic_isFraud_df = pd.DataFrame(synthetic_isFraud_data, columns=column_names)
# Combine the original isFraud data with the synthetic isFraud data
balanced_train_df = pd.concat([train_df[train_df['isFraud'] == 0], synthetic_isFraud_df])


# Convert synthetic isFraud data into a DataFrame
# Assuming column names based on features used
column_names = ['type', 'amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest',
                'isFlaggedFraud', 'balance_change_orig', 'drained_orig_balance',
                'zero_balance_dest', 'zero_balance_orig']

synthetic_isFraud_df = pd.DataFrame(synthetic_isFraud_data, columns=column_names)

# Add the 'isFraud' label to the synthetic data
synthetic_isFraud_df['isFraud'] = 1

# Combine the original non-isFraud data with the synthetic isFraud data
balanced_train_df = pd.concat([train_df[train_df['isFraud'] == 0], synthetic_isFraud_df])

# Shuffle the balanced training dataset
balanced_train_df = balanced_train_df.sample(frac=1).reset_index(drop=True)

# Save the combined DataFrame for later use
balanced_train_df.to_csv('balanced_train_data.csv', index=False)

print("Balanced training data saved successfully.")


NameError: name 'num_synthetic_samples' is not defined

In [ ]:
balanced_train_df.reset_index(drop=True, inplace=True)

In [ ]:
# Save the balanced dataset to a CSV file
output_file_path = 'balanced_train_data.csv'  # Specify your desired file name and path
balanced_train_df.to_csv(output_file_path, index=False)  # Save DataFrame to CSV without row indices
print(f"Balanced dataset saved to {output_file_path}")


Balanced dataset saved to balanced_train_data.csv


In [ ]:
import pandas as pd

# Initialize an empty list to hold the chunks
df_list = []

# Define the column data types to optimize memory usage
dtype = {
    'type': 'category',
    'amount': 'float32',
    'oldbalanceOrg': 'float32',
    'newbalanceOrig': 'float32',
    'nameOrig': 'object',
    'oldbalanceDest': 'float32',
    'newbalanceDest': 'float32',
    'isFraud': 'int8',
    'isFlaggedFraud': 'int8',
    'balance_change_orig': 'float32',
    'drained_orig_balance': 'float32',
    'zero_balance_dest': 'int8',
    'zero_balance_orig': 'int8'
}

# Read the CSV in chunks
for chunk in pd.read_csv('E:\\NTI Graduation\\balanced_train_data.csv', chunksize=100000, dtype=dtype):
    df_list.append(chunk)

# Concatenate all chunks into a single DataFrame
df_final = pd.concat(df_list, ignore_index=True)

# Proceed with further processing
X = df_final.drop('isFraud', axis=1)
y = df_final['isFraud']

# Example: Display the first few rows of the DataFrame
print(df_final.head())

# Continue with your model training and analysis


ParserError: Error tokenizing data. C error: out of memory

In [ ]:
import pandas as pd
df_final = pd.read_csv(r'E:\NTI Graduation\balanced_train_data.csv')
df_final.head()

,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,isFlaggedFraud,balance_change_orig,drained_orig_balance,zero_balance_dest,zero_balance_orig,isFraud
0,-1.269631,0.342006,1.328035,1.436652,-0.174944,-0.001586,-2.490087,-1.145033,-0.859803,-0.702505,0.0
1,-1.269631,0.053981,-0.207478,-0.139539,-0.086687,-0.001586,-1.304037,-1.145033,-0.859803,-0.702505,0.0
2,0.952399,-0.264788,-0.287458,-0.292442,-0.323814,-0.001586,0.169558,0.873337,1.163057,-0.702505,0.0
3,-0.528954,0.575888,-0.226320,-0.292442,-0.296702,-0.001586,1.373718,0.873337,-0.859803,-0.702505,0.0
4,1.693076,0.043398,-0.288716,-0.292442,0.303248,-0.001586,0.144777,0.873337,-0.859803,1.423478,0.0


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# Prepare dataset
#df_final = pd.read_csv(r'E:\NTI Graduation\balanced_train_data.csv')
X = df_final.drop('isFraud', axis=1)
y = df_final['isFraud']

# Apply Standard Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Splitting the data using stratified train/test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Check for NaN values and remove them
if np.isnan(X_train).any() or np.isnan(y_train).any():
    print("NaN values found in features or target variable, removing those rows...")
    X_train = X_train[~np.isnan(y_train)]
    y_train = y_train[~np.isnan(y_train)]

# Check class distribution
print("\nClass distribution in the training set:\n", pd.Series(y_train).value_counts())

# Build and train the Logistic Regression model
logistic_model = LogisticRegression(class_weight='balanced', random_state=42)
logistic_model.fit(X_train, y_train)

# Predict on the test set
y_pred = logistic_model.predict(X_test)

# Evaluation
print("Validation Accuracy:", logistic_model.score(X_test, y_test))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# Feature importance (coefficients)
feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': logistic_model.coef_[0]})
print("\nFeature Importance:\n", feature_importance_df.sort_values(by='Importance', ascending=False))


NaN values found in features or target variable, removing those rows...

Class distribution in the training set:
 isFraud
0.0    4066888
Name: count, dtype: int64


ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 0.0